In [30]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stock.db")
df = pd.read_sql_query(
    """
                        SELECT STOCK_CODE,
                               STOCK_YEAR,
                               STOCK_DATA
                          FROM vcsc_financial_ratio
                         WHERE stock_factor = 'EPS (VND)' and stock_year not in ('TTM Q1/2018')
                         GROUP BY stock_code,
                                  stock_year,
                                  stock_data;           
                        """, conn)
df['STOCK_YEAR'] = pd.to_numeric(df['STOCK_YEAR'])
df['STOCK_DATA'] = df['STOCK_DATA'].str.replace('-', '')
df['STOCK_DATA'] = df['STOCK_DATA'].str.replace(',', '')


df['STOCK_DATA'] = pd.to_numeric(df['STOCK_DATA'])


df['D'] = df.groupby("STOCK_YEAR")["STOCK_DATA"].rank("dense", ascending=False)
df['C'] = pd.qcut(df['STOCK_DATA'], 10, labels=list(range(10)))

df["quantiles"] = df.groupby("STOCK_YEAR")["STOCK_DATA"].transform(pd.qcut,10,list(range(10)))
df.loc[df['STOCK_CODE'] == 'AAA']

,STOCK_CODE,STOCK_YEAR,STOCK_DATA,D,C,quantiles
0,AAA,2008,3466.0,236.0,7,7.0
1,AAA,2009,5716.0,151.0,8,8.0
2,AAA,2010,9114.0,54.0,9,9.0
3,AAA,2011,6614.0,89.0,9,9.0
4,AAA,2012,2785.0,331.0,6,7.0
5,AAA,2013,2775.0,324.0,6,7.0
6,AAA,2014,1186.0,707.0,3,4.0
7,AAA,2015,740.0,851.0,2,3.0
8,AAA,2016,2293.0,418.0,6,7.0
9,AAA,2017,2672.0,403.0,6,7.0


In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stock.db")
df = pd.read_sql_query(
    """
                        SELECT STOCK_CODE,
                               STOCK_YEAR,
                               STOCK_DATA
                          FROM vcsc_financial_ratio
                         WHERE stock_factor = 'EPS (VND)' and stock_year not in ('TTM Q1/2018')
                         GROUP BY stock_code,
                                  stock_year,
                                  stock_data;           
                        """, conn)
df['STOCK_YEAR'] = pd.to_numeric(df['STOCK_YEAR'])

df2 = pd.read_sql_query(
    """
                        select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from vcsc_stock_price;
                        """, conn)

df2['STOCK_DATE'] = pd.to_datetime(df2['STOCK_DATE'], format='%m/%d/%Y')

df2['STOCK_YEAR'] = df2['STOCK_DATE'].dt.year
df2['STOCK_MONTH'] = df2['STOCK_DATE'].dt.month
df2['STOCK_DAY'] = df2['STOCK_DATE'].dt.day

df2['QUARTER'] = df2['STOCK_DATE'].dt.to_period("Q")

gb = df2.groupby([df2['STOCK_CODE'], df2['STOCK_YEAR'], df2['STOCK_MONTH']]).agg({
    'STOCK_CLOSE':
    'first'
})

df3 = df.groupby([df['STOCK_CODE'], df['STOCK_YEAR']]).agg({'STOCK_DATA': 'first'})




result = pd.merge(df2, df, on=['STOCK_CODE', 'STOCK_YEAR'], how='left')

result2 = result.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR', 'STOCK_MONTH']).first()
result2

dfx = df.copy()
dfx.index += 1 
dfx

dfy = df.copy()
dfy.index += 2 
dfy


#results = pd.merge(dfy, dfx, df, left_index=True, right_index=True)
#results

result3 = pd.concat([df, dfx, dfy], axis=1, sort=False)

result

dfg = df2.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'QUARTER']).first()

dfg




#df.pivot(index='stock_code', columns='stock_year', values='stock_data')
#conn.close()

,STOCK_CODE,STOCK_YEAR,STOCK_DATA
1,AAA,2008,"3,466"
2,AAA,2009,"5,716"
3,AAA,2010,"9,114"
4,AAA,2011,"6,614"
5,AAA,2012,"2,785"
6,AAA,2013,"2,775"
7,AAA,2014,"1,186"
8,AAA,2015,740
9,AAA,2016,"2,293"
10,AAA,2017,"2,672"


In [ ]:

ds_price['INDEX'] = range(1, len(ds_price) + 1)
ds_price

ds_price1 = ds_price.copy()
ds_price1['INDEX'] += 1 

ds_price2 = ds_price1.copy()
ds_price2['INDEX'] += 1 

#ds_pricef = pd.concat([ds_price, ds_price1, ds_price2], axis=1, keys='INDEX')
#ds_pricef = pd.merge(ds_price, ds_price1, on=['INDEX'], right_index=True)
ds_pricef = ds_price.merge(ds_price1, on='INDEX', right_index=True).merge(ds_price2, on='INDEX', right_index=True)

ds_pricef